In [1]:
#all our favourite imports
from matplotlib import pyplot as plt
import numpy as np
import pyart
from netCDF4 import num2date
import pytz
import cartopy
import os
#warnings.filterwarnings("ignore")
%matplotlib inline


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



/Users/scollis/anaconda3/envs/aws-radar/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [88]:
def plot_xsapr2(radar, field = 'reflectivity', cmap=None,
               vmin=None, vmax=None, sweep=None, fig=None):
    
    if sweep is None:
        sweep = 0
    
    
    # Lets get some geographical context
    lats = radar.gate_latitude
    lons = radar.gate_longitude

    min_lon = lons['data'].min()
    min_lat = lats['data'].min()
    max_lat = lats['data'].max()
    max_lon = lons['data'].max()

    print('min_lat:', min_lat, ' min_lon:', min_lon, 
          ' max_lat:', max_lat, ' max_lon:', max_lon)

    index_at_start = radar.sweep_start_ray_index['data'][sweep]
    time_at_start_of_radar = num2date(radar.time['data'][index_at_start], 
                                      radar.time['units'])
    GMT = pytz.timezone('GMT')
    local_time = GMT.fromutc(time_at_start_of_radar)
    fancy_date_string = local_time.strftime('%A %B %d at %I:%M %p %Z')
    print(fancy_date_string)
    if fig is None:
        fig = plt.figure(figsize = [15,10])
    display = pyart.graph.RadarMapDisplayCartopy(radar)
    lat_0 = display.loc[0]
    lon_0 = display.loc[1]

    # Main difference! Cartopy forces you to select a projection first!
    projection = cartopy.crs.Mercator(
                    central_longitude=lon_0,
                    min_latitude=min_lat, max_latitude=max_lat)

    start = radar.sweep_start_ray_index['data'][sweep]
    end = radar.sweep_end_ray_index['data'][sweep]
    nrays = end-start
    mid = int(start + nrays/2.)
    print(mid)
    title = str(radar.elevation['data'][mid]) +\
                ' deg X-SAPR2 ' + field.replace('_',' ') + ' \n' + fancy_date_string
 
    display.plot_ppi_map(
        field, sweep, colorbar_flag=False,
        title=title,
        projection=projection,
        min_lon=min_lon, max_lon=max_lon, min_lat=min_lat, max_lat=max_lat,
        vmin=vmin, vmax=vmax, cmap=cmap)

    lb = display._get_colorbar_label(field)
    cb = plt.colorbar(display.plots[0], shrink=.7, aspect=30, pad=0.01)
    cb.set_label(lb)

    # Mark the radar
    display.plot_point(lon_0, lat_0, label_text='X-SAPR2')

    # Plot some lat and lon lines
    gl = display.ax.gridlines(draw_labels=True,
                              linewidth=2, color='gray', alpha=0.5, linestyle='--')
    gl.xlabels_top = False
    gl.ylabels_right = False

In [176]:
def gen_name(odir, radar, field, postfix=None):
    if postfix is None:
        postfix = '.png'
    rad_start_date = num2date(ena_radar.time['data'][0], ena_radar.time['units']) 
    dstr = rad_start_date.strftime('%Y%d%m_%H%M')
    fname = 'xsapr2_ena_quicklook_' + field + '_' + dstr + postfix
    fqn = os.path.join(odir, fname)
    return fqn

In [183]:
def auto_plot(quicklook_directory, radar, field, param_table, sweep=None, postfix=None):
    fig = plt.figure(figsize = [15,10])
    plot_xsapr2(radar, field = field, cmap=param_table[field]['cmap'],
               vmin=param_table[field]['vmin'],
                vmax=param_table[field]['vmax'], sweep = sweep, fig=fig)
    plt.savefig(gen_name(quicklook_directory, radar, field, postfix), dpi=300)
    plt.close(fig)

In [153]:
def ingest_xsapr2(radar):
    #page 44 of https://github.com/scollis/CfRadial/blob/master/docs/CfRadialDoc.v2.0.draft.pdf
    z_name = 'equivalent_reflectivity_factor'
    v_name = 'radial_velocity_of_scatterers_away_from_instrument'
    wth_name = 'doppler_spectrum_width'
    zdr_name = 'log_differential_reflectivity_hv'
    ldr_name = 'log_linear_depolarization_ratio_hv'
    phidp_name = 'differential_phase_hv'
    kdp_name = 'specific_differential_phase_hv'
    rhv_name = 'cross_correlation_ratio_hv' 
    power_name = 'log_power'
    sqi_name = 'normalized_coherent_power'
    zc_name = 'corrected_equivalent_reflectivity_factor'
    vc_name = 'corrected_radial_velocity_of_scatterers_away_from_instrument'
    zdrc_name = 'corrected_log_differential_reflectivity_hv'
    class_name = 'radar_echo_classification'
    snr_name = 'signal_to_noise_ratio'
    pow_name = 'log_power'

    trans_table = {'Z': {'standard_name': z_name, 'name': 'reflectivity'},
                   'UZ': {'standard_name': z_name, 'name': 'uncorrected_reflectivity'},
                  'UZDR1': {'standard_name': zdr_name, 'name': 'uncorrected_differential_reflectivity_1'},
                  'ZDR1': {'standard_name': zdr_name, 'name': 'differential_reflectivity_1'},
                  'CWv': {'standard_name': wth_name, 'name': 'corrected_spectral_width_vertical'},
                  'AZh': {'standard_name': z_name, 'name': 'attenuation_corrected_reflectivity_horizontal'},
                  'Wh': {'standard_name': wth_name, 'name': 'spectral_width_horizontal'},
                  'UWh': {'standard_name': wth_name, 'name': 'un_spectral_width_horizontal'},
                  'UWv': {'standard_name': wth_name, 'name': 'un_spectral_width_vertical'},
                  'UnVh': {'standard_name': vc_name, 'name': 'unfolded_radial_velocity_horizontal'},
                  'SNRv': {'standard_name': snr_name, 'name': 'signal_to_noise_ratio_vertical'},
                  'SIGPOWh' : {'standard_name': pow_name, 'name': 'log_power_horizontal'},
                  'SIGPOWv' : {'standard_name': pow_name, 'name': 'log_power_vertical'},
                  'UPHIDP': {'standard_name': phidp_name, 'name': 'unfolded_differential_phase'},
                  'KDP': {'standard_name': kdp_name, 'name': 'specific_differential_phase'},
                  'AZDR': {'standard_name': zdrc_name, 'name': 'attenuation_corrected_differential_reflectivity'},
                  'RHOHV': {'standard_name': rhv_name, 'name': 'cross_correlation_ratio_hv', 'units': 'unitless'},
                  'ZDR': {'standard_name': zdr_name, 'name': 'differential_reflectivity'},
                  'Wv': {'standard_name': wth_name, 'name': 'spectral_width_vertical'},
                  'Vv': {'standard_name': v_name, 'name': 'radial_velocity_vertical'},
                  'UZv': {'standard_name': z_name, 'name': 'uncorrected_reflectivity_vertical'},
                  'SQIh': {'standard_name': sqi_name, 'name': 'normalized_coherent_power_horizontal', 'units': 'unitless'},
                  'PHIDP': {'standard_name': phidp_name, 'name': 'differential_phase'},
                  'CMAP': {'standard_name': class_name, 'name': 'clutter_map', 'units': 'unitless'},
                  'SNRh': {'standard_name': snr_name, 'name': 'signal_to_noise_ratio_horizontal'},
                  'Vh': {'standard_name': v_name, 'name': 'radial_velocity_horizontal'},
                  'CWh': {'standard_name': wth_name, 'name': 'corrected_spectral_width_horizontal'},
                  'AZDR1': {'standard_name': zdrc_name, 'name': 'attenuation_corrected_differential_reflectivity_1'},
                  'UZh': {'standard_name': z_name, 'name': 'uncorrected_refelctivity_horizontal'},
                  'Zv': {'standard_name': z_name, 'name': 'reflectivity_horizontal_vertical'},
                  'URHOHV':  {'standard_name': rhv_name, 'name': 'uncorrected_cross_correlation_ratio_hv', 'units': 'unitless'},
                  'Zh': {'standard_name': z_name, 'name': 'reflectivity_horizontal'},
                  'CLASS': {'standard_name': class_name, 'name': 'echo_id', 'units': 'unitless'}, 
                  'UZDR': {'standard_name': zdr_name, 'name': 'uncorrected_differential_reflectivity'},
                  'UnVv': {'standard_name': vc_name, 'name': 'unfolded_radial_velocity_vertical'},
                  'SQIv': {'standard_name': sqi_name, 'name': 'normalized_coherent_power_vertical', 'units': 'unitless'}}
    
    for field_name in list(radar.fields.keys()):
        for transfer_item in list(trans_table[field_name].keys()):
            if transfer_item != 'name':
                radar.fields[field_name][transfer_item] = trans_table[field_name][transfer_item]
        radar.fields[field_name]['HDF_name'] = field_name
        radar.fields[trans_table[field_name]['name']] = radar.fields.pop(field_name)
    
    return radar
    

In [157]:
def make_plotting_paramaters(radar):
    maps = pyart.graph.cm
    nyq = radar.instrument_parameters['nyquist_velocity']['data'][0]

    standard_z = {'vmin' : -40, 'vmax' : 40, 'cmap': maps.NWSRef}
    standard_zdr = {'vmin' : -1, 'vmax' : 2, 'cmap': maps.LangRainbow12}
    standard_width = {'vmin' : 0, 'vmax' : nyq/2.0, 'cmap': maps.LangRainbow12}
    standard_snr = {'vmin' : -30, 'vmax' : 30, 'cmap': maps.NWSRef}
    standard_vel = {'vmin' : -nyq, 'vmax' : nyq, 'cmap': maps.NWSVel}
    standard_zto = {'vmin' : 0, 'vmax' : 1, 'cmap': maps.LangRainbow12}
    standard_phidp_180 = {'vmin' : -180, 'vmax' : 180, 'cmap': maps.LangRainbow12}
    standard_snr = {'vmin' : -80, 'vmax' : 10, 'cmap': maps.NWSRef}
    standard_pow = {'vmin' : -5, 'vmax' : 20, 'cmap': maps.NWSRef}


    plotting_table = {'reflectivity': standard_z,
                  'uncorrected_reflectivity': standard_z,
                  'uncorrected_differential_reflectivity_1': standard_zdr,
                  'differential_reflectivity_1': standard_zdr,
                  'corrected_spectral_width_vertical': standard_width,
                  'attenuation_corrected_reflectivity_horizontal': standard_z,
                  'spectral_width_horizontal': standard_width,
                  'unfolded_radial_velocity_horizontal': {'vmin' : -nyq*2.0, 'vmax' : nyq*2.0, 'cmap': maps.NWSVel},
                  'signal_to_noise_ratio_vertical': standard_snr,
                  'unfolded_differential_phase': {'vmin' : 0, 'vmax' : 180, 'cmap': maps.LangRainbow12},
                  'specific_differential_phase': {'vmin' : -1, 'vmax' : 8, 'cmap': maps.LangRainbow12},
                  'attenuation_corrected_differential_reflectivity': standard_zdr,
                  'cross_correlation_ratio_hv': {'vmin' : 0.5, 'vmax' : 1, 'cmap': maps.LangRainbow12},
                  'differential_reflectivity': standard_zdr,
                  'spectral_width_vertical': standard_width,
                  'radial_velocity_vertical': standard_vel,
                  'uncorrected_reflectivity_vertical': standard_z,
                  'normalized_coherent_power_horizontal': standard_zto,
                  'differential_phase': standard_phidp_180,
                  'clutter_map': {'vmin' : 0, 'vmax' : 10, 'cmap': maps.LangRainbow12},
                  'signal_to_noise_ratio_horizontal': standard_snr,
                  'radial_velocity_horizontal': standard_vel,
                  'corrected_spectral_width_horizontal': standard_width,
                  'attenuation_corrected_differential_reflectivity_1': standard_zdr,
                  'uncorrected_refelctivity_horizontal': standard_z,
                  'reflectivity_horizontal': standard_z,
                  'uncorrected_cross_correlation_ratio_hv':  {'vmin' : 0.5, 'vmax' : 1, 'cmap': maps.LangRainbow12},
                  'reflectivity_horizontal_vertical': standard_z,
                  'echo_id': {'vmin' : 0, 'vmax' : 10, 'cmap': maps.LangRainbow12}, 
                  'uncorrected_differential_reflectivity': standard_zdr,
                  'unfolded_radial_velocity_vertical': standard_vel,
                  'normalized_coherent_power_vertical': standard_zto,
                  'log_power_horizontal' : standard_pow,
                  'log_power_vertical' : standard_pow }
    return plotting_table

In [173]:
filename = '/data/xsapr2_good_data/enaxsaprD1.00.20180214.132501.raw.sec_XSAPR2_20180214130010_00.h5'
ena_radar =  ingest_xsapr2(pyart.aux_io.gamic_hdf5.read_gamic(filename, file_field_names=True))
plt_parms = make_plotting_paramaters(ena_radar)

In [174]:
quicklook_directory = '/data/xsapr2_good_data/quicklooks/'

In [184]:
ena_radar.fields['differential_reflectivity_1']['data'] = ena_radar.fields['differential_reflectivity_1']['data'] + 2.2
auto_plot(quicklook_directory, ena_radar, 'reflectivity_horizontal', plt_parms, sweep=3, postfix='.jpg')
auto_plot(quicklook_directory, ena_radar, 'differential_reflectivity_1', plt_parms, sweep=3, postfix='.jpg')
auto_plot(quicklook_directory, ena_radar, 'log_power_horizontal', plt_parms, sweep=3, postfix='.jpg')



min_lat: 39.090954  min_lon: -29.148726  max_lat: 39.98934  max_lon: -26.872225
Wednesday February 14 at 01:01 PM GMT
1130
min_lat: 39.090954  min_lon: -29.148726  max_lat: 39.98934  max_lon: -26.872225
Wednesday February 14 at 01:01 PM GMT
1130
min_lat: 39.090954  min_lon: -29.148726  max_lat: 39.98934  max_lon: -26.872225
Wednesday February 14 at 01:01 PM GMT
1130


In [204]:
happy_x_bands = '/data/xsapr2_good_data/'
all_radars = os.listdir(happy_x_bands)
des = '2018021417'
tb_plot = []
for xfile in all_radars:
    if des in xfile:
        tb_plot.append(happy_x_bands+xfile)

In [205]:
print(tb_plot)

['/data/xsapr2_good_data/enaxsaprD1.00.20180214.182501.raw.sec_XSAPR2_20180214172011_00.h5', '/data/xsapr2_good_data/enaxsaprD1.00.20180214.182501.raw.sec_XSAPR2_20180214175010_00.h5', '/data/xsapr2_good_data/enaxsaprD1.00.20180214.182501.raw.sec_XSAPR2_20180214174010_00.h5', '/data/xsapr2_good_data/enaxsaprD1.00.20180214.172501.raw.sec_XSAPR2_20180214171513_00.h5', '/data/xsapr2_good_data/enaxsaprD1.00.20180214.172501.raw.sec_XSAPR2_20180214170502_00.h5', '/data/xsapr2_good_data/enaxsaprD1.00.20180214.182501.raw.sec_XSAPR2_20180214173509_00.h5', '/data/xsapr2_good_data/enaxsaprD1.00.20180214.182501.raw.sec_XSAPR2_20180214173005_00.h5', '/data/xsapr2_good_data/enaxsaprD1.00.20180214.172501.raw.sec_XSAPR2_20180214171013_00.h5', '/data/xsapr2_good_data/enaxsaprD1.00.20180214.182501.raw.sec_XSAPR2_20180214174510_00.h5', '/data/xsapr2_good_data/enaxsaprD1.00.20180214.182501.raw.sec_XSAPR2_20180214172513_00.h5']


In [ ]:
tb_plot.sort()
for fn in tb_plot:
    ena_radar =  ingest_xsapr2(pyart.aux_io.gamic_hdf5.read_gamic(fn, file_field_names=True))
    plt_parms = make_plotting_paramaters(ena_radar)
    try:
        ena_radar.fields['differential_reflectivity_1']['data'] = ena_radar.fields['differential_reflectivity_1']['data'] + 2.2
        auto_plot(quicklook_directory, ena_radar, 'reflectivity_horizontal', plt_parms, sweep=3, postfix='.jpg')
        auto_plot(quicklook_directory, ena_radar, 'differential_reflectivity_1', plt_parms, sweep=3, postfix='.jpg')
        auto_plot(quicklook_directory, ena_radar, 'log_power_horizontal', plt_parms, sweep=3, postfix='.jpg')
    except:
        print(fn)



min_lat: 39.090954  min_lon: -29.148155  max_lat: 39.98933  max_lon: -26.872124
/data/xsapr2_good_data/enaxsaprD1.00.20180214.172501.raw.sec_XSAPR2_20180214170502_00.h5
min_lat: 39.090954  min_lon: -29.148258  max_lat: 39.989338  max_lon: -26.872087
Wednesday February 14 at 05:11 PM GMT
1129
min_lat: 39.090954  min_lon: -29.148258  max_lat: 39.989338  max_lon: -26.872087
Wednesday February 14 at 05:11 PM GMT
1129
min_lat: 39.090954  min_lon: -29.148258  max_lat: 39.989338  max_lon: -26.872087
Wednesday February 14 at 05:11 PM GMT
1129
min_lat: 39.090954  min_lon: -29.149508  max_lat: 39.989338  max_lon: -26.872433
Wednesday February 14 at 05:16 PM GMT
1125
min_lat: 39.090954  min_lon: -29.149508  max_lat: 39.989338  max_lon: -26.872433
Wednesday February 14 at 05:16 PM GMT
1125
min_lat: 39.090954  min_lon: -29.149508  max_lat: 39.989338  max_lon: -26.872433
Wednesday February 14 at 05:16 PM GMT
1125
min_lat: 39.090954  min_lon: -29.148155  max_lat: 39.989338  max_lon: -26.872187
Wednes